In [1]:
# convert as many ICD10-to-ICD9 1:many mappings as possible into 1:1 mappings
import pandas as pd
import os
cwd = os.getcwd()

In [2]:
# Read the contents of 2018_I10gem.txt
File1    = cwd + '/2018_I10gem.txt'
columns  = ['Code10','Code9','Flags'] 
ICD10To9 = pd.read_fwf(File1, names=columns, converters={0:str, 1:str, 2:str})
ICD10To9.drop('Flags', axis=1, inplace=True) # Remove the Flags column

# Read the contents of ICD9.csv and ICD10.csv
# The reason that I read TotalDiag as string is that the last row contains a big number so I cannot read it as int.
# After reading it as string I need to delete that row and convert it back to int.
ICD9  = pd.read_csv(cwd + '/ICD9.csv',converters={0:str, 0:str})
ICD9.drop(ICD9.columns[2:], axis=1,inplace=True) # Just leave the first two columns. I don't need the rest.
ICD9.drop(ICD9.tail(1).index,inplace=True) #remove the last row
ICD9["TotalDiag"]  = pd.to_numeric(ICD9["TotalDiag"]) #Convert the second column to numeric

GroupedCode = ICD10To9.sort_index().groupby('Code10').filter(lambda group: len(group) != 1) #Find Code10s that are repeated.

# Add a '.' to the Code9 after the third element to match the data in ICD9
def addpoint(code):
    return code[0:3]+'.'+code[3:]

GroupedCode['Code9'] = GroupedCode.Code9.apply(addpoint)

In [3]:
merged_df = pd.merge(GroupedCode, ICD9, right_on='ICD9CMCode', left_on='Code9')

Freq = merged_df['TotalDiag'].tolist()

In [6]:
merged_df.head(10)

,Code10,Code9,ICD9CMCode,TotalDiag
0,A021,003.1,003.1,149
1,A021,995.91,995.91,116538
2,A227,995.91,995.91,116538
3,A267,995.91,995.91,116538
4,A327,995.91,995.91,116538
5,A400,995.91,995.91,116538
6,A401,995.91,995.91,116538
7,A403,995.91,995.91,116538
8,A408,995.91,995.91,116538
9,A409,995.91,995.91,116538


In [9]:
merged_df.info

<bound method DataFrame.info of       Code10   Code9 ICD9CMCode  TotalDiag
0       A021   003.1      003.1        149
1       A021  995.91     995.91     116538
2       A227  995.91     995.91     116538
3       A267  995.91     995.91     116538
4       A327  995.91     995.91     116538
5       A400  995.91     995.91     116538
6       A401  995.91     995.91     116538
7       A403  995.91     995.91     116538
8       A408  995.91     995.91     116538
9       A409  995.91     995.91     116538
10     A4101  995.91     995.91     116538
11     A4102  995.91     995.91     116538
12      A411  995.91     995.91     116538
13      A412  995.91     995.91     116538
14      A413  995.91     995.91     116538
15      A414  995.91     995.91     116538
16     A4150  995.91     995.91     116538
17     A4151  995.91     995.91     116538
18     A4152  995.91     995.91     116538
19     A4153  995.91     995.91     116538
20     A4159  995.91     995.91     116538
21     A4181  995.91  

In [20]:
merged_df.sort_values('TotalDiag', ascending=False).drop_duplicates(['Code10'])[50:100]

,Code10,Code9,ICD9CMCode,TotalDiag
15,A414,995.91,995.91,116538
16,A4150,995.91,995.91,116538
1,A021,995.91,995.91,116538
26,B377,995.91,995.91,116538
25,A5486,995.91,995.91,116538
24,A427,995.91,995.91,116538
22,A4189,995.91,995.91,116538
21,A4181,995.91,995.91,116538
20,A4159,995.91,995.91,116538
19,A4153,995.91,995.91,116538
